In [86]:
! pip install gwpy

In [87]:
! pip install gwpy bilby

In [88]:
!pip install lalsuite


In [89]:
# importing packages
import astropy.constants as ac
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import bilby

In [90]:
from gwpy.timeseries import TimeSeries

# Define GPS time of event and duration (e.g., 4 seconds around GW150914)
event_time = 1126259462.4  # GPS time for GW150914
duration = 4

# Fetch data for both detectors (H1 and L1)
h1_data = TimeSeries.fetch_open_data('H1', event_time - duration / 2, event_time + duration / 2)
l1_data = TimeSeries.fetch_open_data('L1', event_time - duration / 2, event_time + duration / 2)

In [91]:
# Set up priors for the binary black hole merger
priors = bilby.gw.prior.BBHPriorDict()
priors['mass_1'] = bilby.core.prior.Uniform(20, 50, 'mass_1')  # Mass of primary black hole
priors['mass_2'] = bilby.core.prior.Uniform(20, 50, 'mass_2')  # Mass of secondary black hole
priors['luminosity_distance'] = bilby.core.prior.Uniform(100, 2000, 'luminosity_distance')  # Distance
priors['a_1'] = bilby.core.prior.Uniform(0, 0.99, 'a_1')  # Spin of primary
priors['a_2'] = bilby.core.prior.Uniform(0, 0.99, 'a_2')  # Spin of secondary
priors['theta_jn'] = bilby.core.prior.Uniform(0, np.pi, 'theta_jn')  # Inclination
priors['phase'] = bilby.core.prior.Uniform(0, 2 * np.pi, 'phase')
priors['geocent_time'] = bilby.core.prior.Uniform(event_time - 0.1, event_time + 0.1, 'geocent_time')  # Event time
priors['psi'] = bilby.core.prior.Uniform(0, np.pi, 'psi')

17:30 bilby INFO    : No prior given, using default BBH priors in /usr/local/lib/python3.10/dist-packages/bilby/gw/prior_files/precessing_spins_bbh.prior.


In [92]:
# Define waveform arguments
waveform_arguments = {
    'waveform_approximant': 'IMRPhenomPv2',  # Precessing binary black hole waveform
    'reference_frequency': 50.0,
    'minimum_frequency': 20.0
}

In [93]:
duration = 1 * u.s
sampling_frequency = 4096 * u.Hz
interferometers = bilby.gw.detector.InterferometerList(['H1', 'L1'])


  # Define the waveform generator
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    waveform_arguments=waveform_arguments,
)

# Create likelihood
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers=interferometers,
    waveform_generator=waveform_generator,
    time_marginalization=False,
    priors=priors
)

17:30 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [94]:
for ifo, data in zip(interferometers, [h1_data, l1_data]):
    ifo.strain_data = bilby.gw.detector.strain_data.InterferometerStrainData(data.value)

    # Ensure consistent sampling parameters between data and Interferometer
    ifo.strain_data.sampling_frequency = sampling_frequency.value
    ifo.strain_data.duration = duration.value

    # Generate PSD using consistent parameters
    power_spectral_density = data.psd(fftlength=4)

    # Resampling may not be necessary anymore if parameters are consistent
    # If still needed, ensure df is calculated based on the updated sampling_frequency
    df = ifo.strain_data.frequency_array[1] - ifo.strain_data.frequency_array[0]
    power_spectral_density = power_spectral_density.interpolate(df)

    ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
        frequency_array=power_spectral_density.frequencies.value,
        psd_array=power_spectral_density.value
    )

    # Explicitly set duration and sampling_frequency for the Interferometer
    ifo.duration = duration.value
    ifo.sampling_frequency = sampling_frequency.value

In [111]:
# Resample or trim data if needed
min_length = min(len(h1_data), len(l1_data))
h1_data = h1_data[:min_length]
l1_data = l1_data[:min_length]

In [112]:
# Resample to the same rate if necessary
h1_data = h1_data.resample(4096)
l1_data = l1_data.resample(4096)

/usr/local/lib/python3.10/dist-packages/gwpy/timeseries/timeseries.py:930: UserWarning: resample() rate matches current sample_rate (4096.0 Hz), returning input data unmodified; please double-check your parameters
  warnings.warn(


In [113]:
print(h1_data.shape)
print(l1_data.shape)

(16384,)
(16384,)


In [114]:
# Bandpass filter to focus on frequencies where the signal is prominent
h1_data = h1_data.bandpass(20, 1024)
l1_data = l1_data.bandpass(20, 1024)

# Whiten the data to reduce the impact of the instrument noise
h1_data = h1_data.whiten()
l1_data = l1_data.whiten()

In [115]:
# Define priors
priors = bilby.gw.prior.BBHPriorDict()
priors['geocent_time'] = bilby.core.prior.Uniform(
    minimum=h1_data.times.min().value,  # Start time of data
    maximum=h1_data.times.max().value,  # End time of data
    name='geocent_time',
    latex_label='$t_c$',
    unit='$s$'
)

17:53 bilby INFO    : No prior given, using default BBH priors in /usr/local/lib/python3.10/dist-packages/bilby/gw/prior_files/precessing_spins_bbh.prior.


In [116]:
# Set up interferometers with the data from gwpy
interferometers = bilby.gw.detector.InterferometerList(['H1', 'L1'])

waveform_arguments = {
    'waveform_approximant': 'IMRPhenomPv2',
    'reference_frequency': 50,
    'minimum_frequency': 20
}

 # Define the waveform generator
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    waveform_arguments=waveform_arguments,
)

# Create likelihood
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers=interferometers,
    waveform_generator=waveform_generator,
    time_marginalization=False,
    priors=priors
)

17:53 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [128]:
# Set up interferometers with the data from gwpy
interferometers = bilby.gw.detector.InterferometerList(['H1', 'L1'])
interferometers[0].strain_data.set_from_gwpy_timeseries(h1_data)  # Set data for H1
interferometers[1].strain_data.set_from_gwpy_timeseries(l1_data)  # Set data for L1

# Explicitly set the duration and sampling frequency for the interferometers
for interferometer in interferometers:
    interferometer.duration = duration
    interferometer.sampling_frequency = sampling_frequency
    # Instead of implicit truthiness check, explicitly check if the values are not zero
    if sampling_frequency.value != 0 and duration.value != 0:
        interferometer.strain_data._times_and_frequencies.sampling_frequency = sampling_frequency.value
        interferometer.strain_data._times_and_frequencies.duration = duration.value

    # Recalculate the frequency mask after bandpassing
    # Ensure mask length matches frequency_domain_strain length
    # Get the frequency domain strain AFTER setting the data and other parameters
    frequency_domain_strain = interferometer.strain_data.frequency_domain_strain
    mask_length = len(frequency_domain_strain)

    # Create a mask based on the frequency range of the data
    frequencies = interferometer.strain_data.frequency_array  # Access frequency array
    # Create a mask with the same length as the frequency_domain_strain
    mask = np.zeros(mask_length, dtype=bool)
    # Find the indices corresponding to the desired frequency range
    freq_indices = np.where((frequencies >= 20) & (frequencies <= 1024))
    # Set the mask to True for those indices
    mask[freq_indices] = True

    # Apply the mask to the frequency_domain_strain
    interferometer.strain_data.frequency_mask = mask
    # Now frequency_mask has the correct shape


# Run sampler
result = bilby.run_sampler(
    likelihood=likelihood,
    priors=priors,
    sampler='dynesty',
    nlive=500,
    outdir='outdir',
    label='GW_event',
)

18:04 bilby INFO    : Running for label 'GW_event', output will be saved to 'outdir'
18:04 bilby INFO    : Using lal version 7.6.0
18:04 bilby INFO    : Using lal git version Branch: None;Tag: lalsuite-v7.23;Id: c5582baa0aa526a28cb2ddee020dfbee3572be7c;;Builder: Unknown User <>;Repository status: UNCLEAN: Modified working tree
18:04 bilby INFO    : Using lalsimulation version 6.0.0
18:04 bilby INFO    : Using lalsimulation git version Branch: None;Tag: lalsuite-v7.23;Id: c5582baa0aa526a28cb2ddee020dfbee3572be7c;;Builder: Unknown User <>;Repository status: UNCLEAN: Modified working tree
/usr/local/lib/python3.10/dist-packages/bilby/core/utils/log.py:73: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  vdict[key] = str(getattr(sys.modules[key], "__version__", "N/A"))
18:04 bilby INFO    : Analysis priors:
18:04 bilby INFO    : mass_ratio=bilby.gw.prior.UniformInCo

ValueError: operands could not be broadcast together with shapes (8193,) (2049,) 